In [1]:
pip install feather-format

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2458 sha256=92d2bebc7c8dbd6c41ba45a95509bf0edcad8c94a41e1a0a79ee793b7488a9b4
  Stored in directory: c:\users\leonj\appdata\local\pip\cache\wheels\77\5b\0e\0e63d10b6353208a085a321ea2eed2578f220a77bb8a4bd7ab
Successfully built feather-format
Note: you may need to restart the kernel to use updated packages.


In [1]:
import feather 
df = feather.read_dataframe("stk_daily.feather")

In [3]:
daliy_csv = df.to_csv("stk_daily.csv")
daliy_csv

In [26]:
import backtrader as bt
import pandas as pd
import numpy as np

In [18]:
class NDayReversalStrategy(bt.Strategy):
    params = (('n', 5), ('threshold', 0.05),)

    def __init__(self):
        self.order = None
        self.cum_return = bt.indicators.PercentChange(self.data.close, period=self.params.n)

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')  # Print date and log message
        
    def next(self):
        if self.order:
            return

        if not self.position:
            if self.cum_return[0] < -self.params.threshold:
                self.order = self.buy()
        else:
            if self.cum_return[0] > self.params.threshold:
                self.order = self.sell()
        

In [22]:
# Change the format of date to match the required date format that used for cerebro
csv_file = pd.read_csv('stk_daily.csv')
csv_file['date'] = pd.to_datetime(csv_file['date'], format='%d/%m/%Y')
csv_file 

,stk_id,date,open,high,low,close,volume,amount,cumadj
0,000001.SZ,2020-01-02,16.65,16.95,16.55,16.87,153023000,2571200000,98.09860
1,000001.SZ,2020-01-03,16.94,17.31,16.92,17.18,111619000,1914500000,98.09860
2,000001.SZ,2020-01-06,17.01,17.34,16.91,17.07,86208400,1477930000,98.09860
3,000001.SZ,2020-01-07,17.13,17.28,16.95,17.15,72860800,1247050000,98.09860
4,000001.SZ,2020-01-08,17.00,17.05,16.63,16.66,84782400,1423610000,98.09860
...,...,...,...,...,...,...,...,...,...
1048570,003012.SZ,2022-12-20,8.38,8.41,8.07,8.26,9570100,78744700,1.04149
1048571,003012.SZ,2022-12-21,8.31,8.45,8.23,8.42,6222300,51958600,1.04148
1048572,003012.SZ,2022-12-22,8.42,8.73,8.39,8.47,10418300,89204400,1.04149
1048573,003012.SZ,2022-12-23,8.16,8.38,8.09,8.25,8744400,72029300,1.04148


In [23]:
# Define a custom data feed to include 'amount' and 'cumadj'
class CustomCSVData(bt.feeds.PandasData):
    lines = ('amount', 'cumadj',)
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('amount', 'amount'),
        ('cumadj', 'cumadj'),
        ('openinterest', -1),
    )

# Load the CSV data into a DataFrame
dataframe = pd.DataFrame(csv_file)
dataframe['date'] = pd.to_datetime(dataframe['date'])
dataframe.set_index('date', inplace=True)

# Initialize Cerebro
cerebro = bt.Cerebro()

# Add N-Day Reversal Strategy
cerebro.addstrategy(NDayReversalStrategy, n=5, threshold=0.05)

# Add a data feed for each stock
for stk_id in dataframe['stk_id'].unique():
    # Filter the DataFrame for the current stk_id
    stk_data = dataframe[dataframe['stk_id'] == stk_id].copy()
    stk_data.drop('stk_id', axis=1, inplace=True)  # Drop stk_id column as it's no longer needed
    
    # Create and add the data feed
    data_feed = CustomCSVData(dataname=stk_data)
    cerebro.adddata(data_feed, name=stk_id)

# Set broker parameters (optional)
cerebro.broker.set_cash(100000)
cerebro.broker.setcommission(commission=0.001)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Add necessary analyzers for the metrics
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='annual_return')

# Run the strategy
strategies = cerebro.run()
first_strat = strategies[0]

In [30]:
# Analyzers
time_returns = first_strat.analyzers.time_return.get_analysis()
sharpe_ratio = first_strat.analyzers.sharpe.get_analysis()['sharperatio']
drawdown = first_strat.analyzers.drawdown.get_analysis()['max']['drawdown']
annual_returns = first_strat.analyzers.annual_return.get_analysis()

# Calculate annualized volatility
daily_returns = pd.Series([value for key, value in time_returns.items()])
annualized_volatility = daily_returns.std() * np.sqrt(252)  # Assuming 252 trading days in a year

# Excess returns would require a benchmark return series for comparison. 
# If we assume a risk-free rate of return, we can subtract this from our strategy's returns
risk_free_rate = 0.01  # 1% risk-free rate
daily_risk_free_return = risk_free_rate / 252  # Convert to daily return
excess_returns = daily_returns - daily_risk_free_return
annual_excess_returns = (1 + excess_returns).cumprod().iloc[-1]**(252/len(excess_returns)) - 1  # Annualize the excess returns

In [ ]:
# Print the results
print(f"Annualized Return: {list(annual_returns.values())[0]}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"Annualized Volatility: {annualized_volatility}")
print(f"Max Drawdown: {drawdown}")
print(f"Annual Excess Returns: {annual_excess_returns}")

# Plotting the net value curve
cerebro.plot()